In [1]:
import os
import numpy as np
import pandas as pd
import hickle as hkl

from collections import defaultdict

from sklearn.metrics import roc_auc_score, r2_score, accuracy_score, explained_variance_score, mean_absolute_error, mean_squared_error

import keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, Dropout, LSTM, Dense, Concatenate
from tensorflow.keras.callbacks import EarlyStopping

from IPython.core.display import display, HTML



# Hide GPU from visible devices

#'''
tf.config.set_visible_devices([], 'GPU')
print(f'CUDA GPU AVAILABLE: {tf.test.is_gpu_available(cuda_only=True)}')
'''
THREADS = 8
os.environ['OMP_NUM_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTEROP_THREADS'] = str(THREADS)
os.environ['TF_NUM_INTRAOP_THREADS'] = str(THREADS)
tf.config.threading.set_inter_op_parallelism_threads(THREADS)
tf.config.threading.set_intra_op_parallelism_threads(THREADS)
tf.config.set_soft_device_placement(True)
'''

#'''
display(HTML("<style>.container { width:100% !important; }</style>"))
#'''

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
CUDA GPU AVAILABLE: True


In [2]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean((y_pred - y_true) / y_true)

In [3]:
N_SPLITS = 5
N_STEPS = 20
N_DAYS = 60
DATA_DIR = f'data/processed_data/cv_data__{N_SPLITS}_class_id_folds__{N_STEPS}_steps__{N_DAYS}_days'

In [4]:
results = []

for i in range(N_SPLITS):

    # Load one partition of folds
    data = hkl.load(f'{DATA_DIR}/fold_{i}.hkl')
    action_training_input = data['action_training_input']
    #recurrent_training_input = data['recurrent_training_input']
    #prior_training_input = data['prior_training_input']
    completion_training_target = data['completion_training_target']
    problems_training_target = data['problems_training_target']
    training_target_sequence = data['training_target_sequence']
    action_testing_input = data['action_testing_input']
    #recurrent_testing_input = data['recurrent_testing_input']
    #prior_testing_input = data['prior_testing_input']
    completion_testing_target = data['completion_testing_target']
    problems_testing_target = data['problems_testing_target']
    testing_target_sequence = data['testing_target_sequence']
    
    # Clear session so models don't pile up
    keras.backend.clear_session()

    # Create model
    
    # Action Model
    action_input_layer = Input(shape=action_training_input[0].shape, name='action')
    action_model = Dropout(rate=0.5)(action_input_layer)
    action_model = LSTM(units=64, return_sequences=False, activation='relu', dropout=0.5, recurrent_dropout=0.5)(action_model)
    
    # Recurrent Input
    #recurrent_input_layer = Input(shape=recurrent_training_input[0].shape, name='recurrent')
    #recurrent_model = Masking(mask_value=0.0)(recurrent_input_layer)
    #recurrent_model = Dropout(rate=0.5)(recurrent_model)
    #recurrent_model = LSTM(units=64, return_sequences=False, dropout=0.5, recurrent_dropout=0.5)(recurrent_model)
    
    # Prior Input
    #prior_input_layer = Input(shape=prior_training_input[0].shape, name='prior')
    #prior_model = Dropout(rate=0.5)(prior_input_layer)
    #prior_model = Dense(units=64, activation='sigmoid')(prior_model)
    
    # Combined Model
    model = Concatenate()([action_model])
    model = Dropout(rate=0.5)(action_model)
    
    # Combined Output
    completion_output_layer = Dense(units=1, activation='sigmoid', name='completion')(model)
    problems_output_layer = Dense(units=1, activation='linear', name='problems')(model)
    
    combined_model = Model(action_input_layer, [completion_output_layer, problems_output_layer])
    combined_model.compile(optimizer='adam', loss={'completion': 'binary_crossentropy', 'problems': 'mse'})

    # Train model
    es = [EarlyStopping(monitor='val_loss', patience=10, min_delta=0, restore_best_weights=True)]
    weights = {'completion': np.ones_like(completion_training_target) * 16, 'problems': completion_training_target}
    combined_model.fit(x={'action': action_training_input},
                       y={'completion': completion_training_target, 'problems': problems_training_target},
                       epochs=1000,
                       validation_split=0.25,
                       callbacks=es,
                       sample_weight=weights,
                       verbose=1)
    
    # Store model predictions
    completion_testing_output, problems_testing_output = combined_model.predict({'action': action_testing_input})
    df = pd.DataFrame(zip(np.ones_like(testing_target_sequence).flatten() * i, 
                          testing_target_sequence.flatten(), 
                          completion_testing_target.flatten(), 
                          problems_testing_target.flatten(), 
                          completion_testing_output.flatten(), 
                          problems_testing_output.flatten()), 
                      columns = ['fold', 
                                 'target_sequence', 
                                 'completion_target', 
                                 'problems_target', 
                                 'completion_prediction', 
                                 'problems_prediction'])
    results.append(df)
    

pd.concat(results).to_csv('actions_cross_validation_results.csv', index=False)

Epoch 1/1000
3949/3949 [==============================] - 332s 84ms/step - loss: 141.1011 - completion_loss: 9.3242 - problems_loss: 131.7769 - val_loss: 17.1153 - val_completion_loss: 9.7797 - val_problems_loss: 7.3356
Epoch 2/1000
3949/3949 [==============================] - 288s 73ms/step - loss: 16.5037 - completion_loss: 8.8369 - problems_loss: 7.6668 - val_loss: 15.7691 - val_completion_loss: 9.7866 - val_problems_loss: 5.9826
Epoch 3/1000
3949/3949 [==============================] - 264s 67ms/step - loss: 15.8640 - completion_loss: 8.8174 - problems_loss: 7.0466 - val_loss: 15.6472 - val_completion_loss: 9.7483 - val_problems_loss: 5.8989
Epoch 4/1000
3949/3949 [==============================] - 254s 64ms/step - loss: 15.7651 - completion_loss: 8.7944 - problems_loss: 6.9707 - val_loss: 15.6037 - val_completion_loss: 9.7210 - val_problems_loss: 5.8827
Epoch 5/1000
3949/3949 [==============================] - 229s 58ms/step - loss: 15.6751 - completion_loss: 8.7423 - problems_los

3949/3949 [==============================] - 268s 68ms/step - loss: 15.3566 - completion_loss: 8.8490 - problems_loss: 6.5076 - val_loss: 15.3860 - val_completion_loss: 9.3008 - val_problems_loss: 6.0851
Epoch 3/1000
3949/3949 [==============================] - 271s 69ms/step - loss: 15.8832 - completion_loss: 8.8217 - problems_loss: 7.0615 - val_loss: 15.4909 - val_completion_loss: 9.4573 - val_problems_loss: 6.0336
Epoch 4/1000
3949/3949 [==============================] - 271s 69ms/step - loss: 15.1181 - completion_loss: 8.7751 - problems_loss: 6.3430 - val_loss: 15.2720 - val_completion_loss: 9.2969 - val_problems_loss: 5.9751
Epoch 5/1000
3949/3949 [==============================] - 270s 68ms/step - loss: 15.0158 - completion_loss: 8.7309 - problems_loss: 6.2849 - val_loss: 15.2336 - val_completion_loss: 9.2769 - val_problems_loss: 5.9566
Epoch 6/1000
3949/3949 [==============================] - 241s 61ms/step - loss: 14.9210 - completion_loss: 8.6825 - problems_loss: 6.2385 - val_

Epoch 4/1000
3949/3949 [==============================] - 223s 56ms/step - loss: 15.7673 - completion_loss: 8.8044 - problems_loss: 6.9629 - val_loss: 15.4998 - val_completion_loss: 9.7350 - val_problems_loss: 5.7647
Epoch 5/1000
3949/3949 [==============================] - 223s 56ms/step - loss: 15.6431 - completion_loss: 8.7573 - problems_loss: 6.8858 - val_loss: 15.4119 - val_completion_loss: 9.6646 - val_problems_loss: 5.7473
Epoch 6/1000
3949/3949 [==============================] - 221s 56ms/step - loss: 15.5512 - completion_loss: 8.6923 - problems_loss: 6.8589 - val_loss: 15.2207 - val_completion_loss: 9.5263 - val_problems_loss: 5.6944
Epoch 7/1000
3949/3949 [==============================] - 223s 56ms/step - loss: 15.4569 - completion_loss: 8.6225 - problems_loss: 6.8344 - val_loss: 15.3251 - val_completion_loss: 9.6305 - val_problems_loss: 5.6945
Epoch 8/1000
3949/3949 [==============================] - 222s 56ms/step - loss: 15.3931 - completion_loss: 8.5778 - problems_loss: 

3949/3949 [==============================] - 231s 59ms/step - loss: 15.1703 - completion_loss: 8.3946 - problems_loss: 6.7757 - val_loss: 14.9263 - val_completion_loss: 9.2829 - val_problems_loss: 5.6434


In [5]:
# Load the data

exp_norm_map = pd.read_csv('data/experiment_information/exp_norm_map.csv')
results = pd.read_csv('actions_cross_validation_results.csv')

In [6]:
# Evaluate the results

norm_exp_dict = defaultdict(lambda: 'None')
for n, e in zip(exp_norm_map['normal_id'], exp_norm_map['experiment_id']):
    norm_exp_dict[n] = e
results['experiment_sequence'] = results['target_sequence'].map(norm_exp_dict)

metrics = []
0
# Calculate the metrics for each sequence
for sequence, df in results.groupby('target_sequence'):
    
    completion_target = df['completion_target']
    
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([sequence, 
                    norm_exp_dict[sequence], 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for each fold
for fold, df in results.groupby('fold'):
    
    completion_target = df['completion_target']
    completion_prediction = df['completion_prediction']
    completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
    completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
    completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
    completion_mse = mean_squared_error(completion_target, completion_prediction)
    
    problems_target = df[df['completion_target'] == 1]['problems_target']
    problems_prediction = df[df['completion_target'] == 1]['problems_prediction']
    problems_mae = mean_absolute_error(problems_target, problems_prediction)
    problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
    problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
    problems_mse = mean_squared_error(problems_target, problems_prediction)

    metrics.append([f'fold_{fold}', 
                    'None', 
                    len(df), 
                    completion_auc, 
                    completion_acc, 
                    completion_r2, 
                    completion_ev, 
                    completion_mse, 
                    problems_mae, 
                    problems_mape, 
                    problems_r2, 
                    problems_ev, 
                    problems_mse])

# Calculate the metrics for everything combined
completion_target = results['completion_target']
completion_prediction = results['completion_prediction']
completion_auc = roc_auc_score(completion_target, completion_prediction) if len(completion_target.unique()) > 1 else None
completion_acc = accuracy_score(completion_target, completion_prediction > 0.5)
completion_r2 = r2_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_ev = explained_variance_score(completion_target, completion_prediction) if len(completion_target) > 1 else None
completion_mse = mean_squared_error(completion_target, completion_prediction)

problems_target = results[results['completion_target'] == 1]['problems_target']
problems_prediction = results[results['completion_target'] == 1]['problems_prediction']
problems_mae = mean_absolute_error(problems_target, problems_prediction)
problems_mape = mean_absolute_percentage_error(problems_target, problems_prediction)
problems_r2 = r2_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_ev = explained_variance_score(problems_target, problems_prediction) if len(problems_target) > 1 else None
problems_mse = mean_squared_error(problems_target, problems_prediction)

metrics.append(['all_target_data', 
                'None', 
                len(results), 
                completion_auc, 
                completion_acc, 
                completion_r2, 
                completion_ev, 
                completion_mse, 
                problems_mae, 
                problems_mape, 
                problems_r2, 
                problems_ev, 
                problems_mse])

metrics = pd.DataFrame(metrics, 
                       columns=['sequence_id', 
                                'experiment_id', 
                                'sample_size', 
                                'completion_auc', 
                                'completion_acc', 
                                'completion_r2', 
                                'completion_ev', 
                                'completion_mse', 
                                'problems_mae', 
                                'problems_mape', 
                                'problems_r2', 
                                'problems_ev', 
                                'problems_mse'])

metrics.to_csv('actions_cross_validation_metrics.csv', index=False)

In [7]:
metrics.iloc[-6:]

,sequence_id,experiment_id,sample_size,completion_auc,completion_acc,completion_r2,completion_ev,completion_mse,problems_mae,problems_mape,problems_r2,problems_ev,problems_mse
288,fold_0,None,42114,0.676557,0.750653,0.047832,0.048065,0.178265,1.951298,0.221988,0.010073,0.010308,7.493673
289,fold_1,None,42114,0.667185,0.744337,0.052847,0.054112,0.180309,1.964423,0.188749,0.005641,0.007681,9.488640
290,fold_2,None,42114,0.675337,0.745809,0.047258,0.052761,0.180697,2.001229,0.208084,0.009780,0.010509,10.142673
291,fold_3,None,42113,0.675707,0.741291,0.060013,0.061634,0.180549,1.958050,0.221717,0.013059,0.013173,7.782712
292,fold_4,None,42113,0.672513,0.746040,0.049394,0.053921,0.180825,1.994662,0.228579,0.008050,0.008254,8.072355
293,all_target_data,None,210568,0.669197,0.745626,0.051551,0.053717,0.180129,1.973921,0.213829,0.009338,0.009397,8.595461
